## Phase 1: Data extraction

https://news.ycombinator.com/item?id=41614126

https://github.com/conjuncts/gmft

https://www.reddit.com/r/LocalLLaMA/comments/1854d06/table_extraction_from_pdf/ ==> some guy did https://docs.google.com/spreadsheets/d/12IhxHZbYF71dPl32PQpF_6pg9e9S8f9W4sTHt-B0KTg/edit?gid=0#gid=0 


https://www.getmagical.com/blog/extract-table-from-pdf


As to whether traditional OCR or LLMs are better for this task

#### Conclusions:
- We REALLY want to keep the right figures, so potentially trying lower end models for this, while it can be accurate a fair amount of the time, likely does not counteract the risk of it eventually hallucinating. Not all LLMs are equal.
- [Marker](https://github.com/vikParuchuri/marker) has 20k stars on github and was found from the excel sheet; but has a commercial usage agreement so Nahh

https://github.com/conjuncts/gmft ? => Relies on Table transformers (microsoft), small, seems ideal. I've also seen stuff for AWS textract but let's keep it simple for now.

In [1]:
import gmft
from gmft.auto import AutoTableDetector, AutoTableFormatter
from gmft.pdf_bindings import PyPDFium2Document
import pandas as pd

# Load PDF and extract tables
pdf_path = "./pdf_inputs/fwc_sample_financial_statement 1.pdf"
detector = AutoTableDetector()
formatter = AutoTableFormatter()

doc = PyPDFium2Document(pdf_path)
tables = []

for page_num, page in enumerate(doc):
    extracted_tables = detector.extract(page)
    for table_index, table in enumerate(extracted_tables):
        formatted_table = formatter.extract(table)
        table_data = formatted_table.df()
        if table_data is not None:
            tables.append(table_data)

doc.close()
print(doc, tables)


<gmft.pdf_bindings.pdfium.PyPDFium2Document object at 0x00000230A3FCB880> [  Motor vehicle expenses Donations Legal costs Administration expenses  \
0                     4H        4G          4F                      4E   
1                 89,000    77,000     296,000               1,649,000   
2                 79,000   120,000      62,000               1,475,000   

  Depreciation Affiliation fees Capitation fees Expenses Employee expenses  \
0           4D               4C              4B                         4A   
1       97,000          228,000         830,000                  3,167,000   
2       88,000          231,000         823,000                  3,023,000   

  revenue      Total Other revenue Rental income Interest  \
0    None       None            3A          None     None   
1    None  7,797,000       613,000       185,000  251,000   
2    None  7,701,000       655,000       244,000  231,000   

  Membership subscriptions          Revenue  \
0                     N

> Tried gmft and it has all these nice metrics etc but when I actually used it, it was terrible - because after some diving in it seems to be designed for actual tables in scientific journals etc, whilst this pdf is amajority of white space, tables go over multiple pages etc so it really does not deal well with this.

Would be way better to just get a text reader for this case
It also takes very long

In [ ]:
import pandas as pd # Sideventure of trying to just throw the pdf at gemini and telling it to generate a csv (DID NOT WORK AT ALL)

# Replace 'your_file.csv' with the actual name of your CSV file
df = pd.read_csv('gemini_test.csv') # wow gemini CANNOT make csvs LOL okay looks like this manual reading off pdf is better atleast.

# Now you can work with the data in the DataFrame 'df'
print(df)

                                                Statement  Item   Category  \
Income                                            Revenue   NaN        NaN   
NaN                              Membership subscriptions   NaN  6748000.0   
NaN                                              Interest   NaN   251000.0   
NaN                                         Rental income   NaN   185000.0   
NaN                                         Other revenue   NaN   613000.0   
...                                                   ...   ...        ...   
NaN                                       Total Cash Used   NaN        NaN   
NaN                    NET CASH FROM FINANCING ACTIVITIES   NaN  8000000.0   
NaN                  NET INCREASE/(DECREASE) IN CASH HELD   NaN  1862000.0   
NaN     Cash and cash equivalents at the beginning of ...   NaN  4955000.0   
NaN     Cash and cash equivalents at the end of the re...   NaN  3093000.0   

        Last Year  Previous Year  
Income        NaN           

In [18]:
import PyPDF2
pdf_path = "fwc_sample_financial_statement 1.pdf"


def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
            text += "\n==================\n"
    return text

pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)

   
FS 023 Sample financial statement   10 March  2023  | p. 1 Fact Sheet  FS 023 | 10 March  2023  
 
Sam ple financial statements  
 
Statement of  comprehensive income  
For the year ended 30 June  
  
Notes  Last Year  
$ Previous Year  
$ 
Revenue     
Membership subscriptions   6,748,000  6,571,000  
Interest   251,000  231,000  
Rental income   185,000  244,000  
Other revenue  3A 613,000  655,000  
Total revenue   7,797,000  7,701,000  
    
Expenses     
Employee expenses  4A 3,167,000  3,023,000  
Capitation fees  4B 830,000  823,000  
Affiliation fees  4C 228,000  231,000  
Depreciation  4D 97,000  88,000  
Administration expenses  4E 1,649,000  1,475,000  
Legal costs  4F 296,000  62,000  
Donations  4G 77,000  120,000  
Motor vehicle expenses  4H 89,000  79,000  

   
FS 023 Sample financial statement   10 March  2023  | p. 2 Other expenses  4I 835,000  775,000  
Total expenses   7,268,000  6,676,000  
    
Surplus (deficit) for the year   529,000  1,025,000  
    
Other c

In [2]:
import pdfplumber # pdfplumber IS SLOWER BTW by a fair bit, but it also does give much nicer formatting.
pdf_path = "./pdf_inputs/fwc_sample_financial_statement 1.pdf"

def extract_full_text(pdf_path):
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            full_text += page.extract_text() + "\n"  # Append text from each page and add a newline
    return full_text

# Extract all text without caring about page numbers
full_text = extract_full_text(pdf_path)

# Print the entire text
print(full_text)

## Per page version below 
# def extract_pdf_by_pages(pdf_path):
#     pages_text = {}
#     with pdfplumber.open(pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages, 1):
#             pages_text[f"page_{page_num}"] = page.extract_text()
#     return pages_text

# # Extract all pages
# pages = extract_pdf_by_pages(pdf_path)

# # Print each page to see what we're working with
# for page_num, text in pages.items():
#     print(page_num, text)

Fact Sheet FS 023 | 10 March 2023
Sample financial statements
Statement of comprehensive income
For the year ended 30 June
Last Year Previous Year
Notes $ $
Revenue
Membership subscriptions 6,748,000 6,571,000
Interest 251,000 231,000
Rental income 185,000 244,000
Other revenue 3A 613,000 655,000
Total revenue 7,797,000 7,701,000
Expenses
Employee expenses 4A 3,167,000 3,023,000
Capitation fees 4B 830,000 823,000
Affiliation fees 4C 228,000 231,000
Depreciation 4D 97,000 88,000
Administration expenses 4E 1,649,000 1,475,000
Legal costs 4F 296,000 62,000
Donations 4G 77,000 120,000
Motor vehicle expenses 4H 89,000 79,000
FS 023 Sample financial statement 10 March 2023 | p. 1
Other expenses 4I 835,000 775,000
Total expenses 7,268,000 6,676,000
Surplus (deficit) for the year 529,000 1,025,000
Other comprehensive income
Gain on revaluation – investment property - 342,000
Total comprehensive income for the year 529,000 1,367,000
The above statement should be read in conjunction with the not

#### Conclusions
PDF plumber takes (noticeably) longer, but is more accurate/has less missing characters so we'll be using this because ideally, the data extraction script should only run once (and we can save the outputs to a DB or something eventually... (foreshadowing: RAG yay!))

In [3]:
import re
### First, let's just clean the headers/footers since those are absolutely useless (And the Notes $ $ )

def clean_text(raw_text):
    # Remove headers, footers, page numbers
    clean = re.sub(r"FS 023.*?\|\s+p\.\s+\d+", "", raw_text)

    # Get rid of end of document text
    clean = re.sub(r'© Commonwealth of Australia.*?legal advice\.', '', clean, flags=re.DOTALL) # DOTALL allows the match to continue across multiple lines.

    return clean

cleaned_text = clean_text(full_text) 
print(cleaned_text)

Fact Sheet FS 023 | 10 March 2023
Sample financial statements
Statement of comprehensive income
For the year ended 30 June
Last Year Previous Year
Notes $ $
Revenue
Membership subscriptions 6,748,000 6,571,000
Interest 251,000 231,000
Rental income 185,000 244,000
Other revenue 3A 613,000 655,000
Total revenue 7,797,000 7,701,000
Expenses
Employee expenses 4A 3,167,000 3,023,000
Capitation fees 4B 830,000 823,000
Affiliation fees 4C 228,000 231,000
Depreciation 4D 97,000 88,000
Administration expenses 4E 1,649,000 1,475,000
Legal costs 4F 296,000 62,000
Donations 4G 77,000 120,000
Motor vehicle expenses 4H 89,000 79,000

Other expenses 4I 835,000 775,000
Total expenses 7,268,000 6,676,000
Surplus (deficit) for the year 529,000 1,025,000
Other comprehensive income
Gain on revaluation – investment property - 342,000
Total comprehensive income for the year 529,000 1,367,000
The above statement should be read in conjunction with the notes.

Statement of financial position
As at 30 June
Las

In [16]:
#### So, each main table section ends with "The above statement should be read in conjunction with the notes."

### In this specific case we COULD just do cleaned_text.split('the notes string thing') but I'll use regex just so we can account for a bit more variation in different pdfs.


def split_into_sections_regex(cleaned_text):
    """Splits cleaned financial statement text into 4 sections with flexible matching of the stop phrase."""
    
    # Regex pattern to match variations like:
    # - "The above statement should be read in conjunction with the notes."
    # - "Above statement must be considered along with the notes."
    # - "above statement should be read carefully with the notes."
    stop_pattern = r"(?:The\s*)?above statement .*? with the notes\.?"

    # Split using regex
    sections = re.split(stop_pattern, cleaned_text, flags=re.IGNORECASE)

    # Remove extra whitespace and filter empty sections
    sections = [sec.strip() for sec in sections if sec.strip()]

    if len(sections) != 4:
        print(f"Warning: Expected 4 sections, but found {len(sections)}")

    return sections

# Apply function
sections = split_into_sections_regex(cleaned_text)
segmented_tables = split_into_sections_regex(cleaned_text)

# Assign sections
income_statement_text, balance_sheet_text, equity_changes_text, cash_flow_text = sections

# Display output
print("\n--- Income Statement ---\n", income_statement_text)
# print("\n--- Balance Sheet ---\n", balance_sheet_text)
# print("\n--- Equity Changes ---\n", equity_changes_text)
# print("\n--- Cash Flow ---\n", cash_flow_text)



--- Income Statement ---
 Fact Sheet FS 023 | 10 March 2023
Sample financial statements
Statement of comprehensive income
For the year ended 30 June
Last Year Previous Year
Notes $ $
Revenue
Membership subscriptions 6,748,000 6,571,000
Interest 251,000 231,000
Rental income 185,000 244,000
Other revenue 3A 613,000 655,000
Total revenue 7,797,000 7,701,000
Expenses
Employee expenses 4A 3,167,000 3,023,000
Capitation fees 4B 830,000 823,000
Affiliation fees 4C 228,000 231,000
Depreciation 4D 97,000 88,000
Administration expenses 4E 1,649,000 1,475,000
Legal costs 4F 296,000 62,000
Donations 4G 77,000 120,000
Motor vehicle expenses 4H 89,000 79,000

Other expenses 4I 835,000 775,000
Total expenses 7,268,000 6,676,000
Surplus (deficit) for the year 529,000 1,025,000
Other comprehensive income
Gain on revaluation – investment property - 342,000
Total comprehensive income for the year 529,000 1,367,000


One thing we'll note here is that certain tables (like table 3) are annoying!
Because they have table headers like:

Asset

Revaluation Retained

Reserve Earnings Total

$ $ $

And the way to read this is:
                "Asset revaluation reserve $" "Retained Earnings $" "Total" 
Balance as at ...
...

I wonder if we should go for a generalised function that does any table, or manually do each table here...?
Again, we **COULD** also just try throwing the processed strings into the LLM? Maybe we'll try that

##### Manually trying to implement dataframes

In [17]:
segmented_tables[0].split('\n')

['Fact Sheet FS 023 | 10 March 2023',
 'Sample financial statements',
 'Statement of comprehensive income',
 'For the year ended 30 June',
 'Last Year Previous Year',
 'Notes $ $',
 'Revenue',
 'Membership subscriptions 6,748,000 6,571,000',
 'Interest 251,000 231,000',
 'Rental income 185,000 244,000',
 'Other revenue 3A 613,000 655,000',
 'Total revenue 7,797,000 7,701,000',
 'Expenses',
 'Employee expenses 4A 3,167,000 3,023,000',
 'Capitation fees 4B 830,000 823,000',
 'Affiliation fees 4C 228,000 231,000',
 'Depreciation 4D 97,000 88,000',
 'Administration expenses 4E 1,649,000 1,475,000',
 'Legal costs 4F 296,000 62,000',
 'Donations 4G 77,000 120,000',
 'Motor vehicle expenses 4H 89,000 79,000',
 '',
 'Other expenses 4I 835,000 775,000',
 'Total expenses 7,268,000 6,676,000',
 'Surplus (deficit) for the year 529,000 1,025,000',
 'Other comprehensive income',
 'Gain on revaluation – investment property - 342,000',
 'Total comprehensive income for the year 529,000 1,367,000']

In [18]:
import pandas as pd

## THIS ONE IS PRETTY CINEMA NO!??
def extract_income_statement(text):
    """Extracts Revenue, Expenses, and Profit from Income Statement."""
    income_statement = []


    pattern = r"""
        ([A-Za-z\s–]+)        # Item name: letters, spaces, dashes
        \s+                   # One or more spaces
        (?:\d[A-Z])?          # Optional Notes column (e.g., 3A)
        \s*                   # Zero or more spaces
        ([\d,]+|-)            # Last Year value: numbers with commas or a dash
        \s+                   # One or more spaces
        ([\d,]+|-)            # Previous Year value: numbers with commas or a dash
    """


    for line in text.split('\n'):
        # Look for lines with amounts
        print(line)
        match = re.search(pattern, line, re.VERBOSE)
        if match:
            item = match.group(1).strip()
            last_year = match.group(2).replace(",", "")
            prev_year = match.group(3).replace(",", "")

            # Convert to numeric (handling "-" as zero)
            last_year = 0 if last_year == "-" else float(last_year)
            prev_year = 0 if prev_year == "-" else float(prev_year)

            income_statement.append({"Item": item, "Last Year": last_year, "Previous Year": prev_year})
    
    return pd.DataFrame(income_statement)

income_statement_df = extract_income_statement(segmented_tables[0])

print("\nIncome Statement:")
display(income_statement_df)

Fact Sheet FS 023 | 10 March 2023
Sample financial statements
Statement of comprehensive income
For the year ended 30 June
Last Year Previous Year
Notes $ $
Revenue
Membership subscriptions 6,748,000 6,571,000
Interest 251,000 231,000
Rental income 185,000 244,000
Other revenue 3A 613,000 655,000
Total revenue 7,797,000 7,701,000
Expenses
Employee expenses 4A 3,167,000 3,023,000
Capitation fees 4B 830,000 823,000
Affiliation fees 4C 228,000 231,000
Depreciation 4D 97,000 88,000
Administration expenses 4E 1,649,000 1,475,000
Legal costs 4F 296,000 62,000
Donations 4G 77,000 120,000
Motor vehicle expenses 4H 89,000 79,000

Other expenses 4I 835,000 775,000
Total expenses 7,268,000 6,676,000
Surplus (deficit) for the year 529,000 1,025,000
Other comprehensive income
Gain on revaluation – investment property - 342,000
Total comprehensive income for the year 529,000 1,367,000

Income Statement:


,Item,Last Year,Previous Year
0,Membership subscriptions,6748000.0,6571000.0
1,Interest,251000.0,231000.0
2,Rental income,185000.0,244000.0
3,Other revenue,613000.0,655000.0
4,Total revenue,7797000.0,7701000.0
5,Employee expenses,3167000.0,3023000.0
6,Capitation fees,830000.0,823000.0
7,Affiliation fees,228000.0,231000.0
8,Depreciation,97000.0,88000.0
9,Administration expenses,1649000.0,1475000.0


Key segments of financial data ARE:

- Income Statement (Revenue, Expenses, Profit)
- Balance Sheet / Financial Position (Assets, Liabilities, Equity)
- Statement of Changes in Equity
- Cash Flow Statement

**We're going to use a way which ASSUMES that pdfs given will follow similarly worded formatting**

# Phase 3 - GENAI Models Yay!!!
Leverage Existing Models:
You might choose to fine-tune or prompt an existing language model (e.g., GPT-4) to further interpret these segments. The goal is to have the model:

- Identify and extract key metrics from each section.
- Understand the context and handle long documents efficiently.


Prompt Engineering: Design prompts that guide the model to extract numbers accurately.



####    I SEE THE PRIVACY ISSUES NOT TO WORRY. -- Sending data to OPENAI etc... Local models!?!?
So, in the case of private information looking into local models would definitely be an idea, but... I have 4GB vram on my PC.


In [ ]:
import openai

def prompt_financial_extraction(section_text):
    prompt = (
        "Extract the following financial metrics from the text: "
        "Revenue, Net Income, Operating Expenses, and Cash Flow. "
        "Return the data in JSON format. Text:\n" + section_text
    )
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=250
    )
    return response.choices[0].text.strip()

# Example usage:
# json_data = prompt_financial_extraction(sections.get('Statement of comprehensive income', ''))


I would try to use local models but unfortunately my PC has spec limitations so :(
    ... IS WHAT I WOULD SAY IF I WASN'T TAKING THIS SERIOUSLY! Jkjk, but seriously my GPU has 4GB vram which is horrendous, so I can try to run very small models but I don't think they're that trustworthy -- I'll do it anyways but yeah..

## Phase 5 - Scaling And Stuff

#### For scaling preprocessing

We currently use a mix of manutal table names w/ regex patterns which is fine here, but in more varied datasets we may have some errors as people use slightly different words/terminology that falls through the regex patterns. So potentially using NLP based methods (like spacy, etc.) would help here. 

(Ideally, the pdfs we'd have aren't spaced messes and actually have concise tables so we can use gmft but ideals do not always happen).


#### For RAG:

Potential for RAG for greater reliability or some sort of pre-processed database so the PDF reading and processing doesn't have to happen every time.

Then you RAG it to get more accurate answers while still being able to ask the GenAI model any query.

RAG is a trade-off of speed for accuracy, but for large databases, it’s pretty practical because of context limits (i.e., you can’t throw 10,000 docs into the LLM, so use a retriever to find 5 relevant documents instead!).

// Optimise retrieval via pre-indexing statements

// Deploying on cloud (AWS lambda) for better pipelines and scalability (e.g preprocessing function runs whenever new pdf is inputted, then thrown into a 
database)



//// https://gemini.google.com/app/92f6d574bc081ed7 ====> for other scaling ideas like docker etc